In [1]:
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing,Holt
#Working on plotly
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
cf.go_offline()
import pandas as pd
from scipy.stats import kurtosis
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [2]:
# Import the file
df=pd.read_csv(r"C:\Users\2000139\TORQ_ptop_1.txt")
df.head()

,Number of points: 1029419
0,Sampling rate(Hz): 0.0166666666666667
1,Time range: From: 05/10/2016 08:44:54; Until: ...
2,Units: kNm
3,01.06.2016 00:00:13\t20.7
4,01.06.2016 00:01:13\t21.5


In [3]:
#Preprocessing steps
col_name='TORQ_ptop'
print(col_name)
df.columns=[col_name]
df.head()
df = pd.DataFrame(df[col_name].str.split('\t',2).tolist())
df.dropna(inplace=True)
df[0]=pd.to_datetime(df[0],format='%d.%m.%Y %H:%M:%S')
df.index=df[0]
df.drop(columns=[0],inplace=True)
df.columns=[col_name]
df.isnull().sum()
df=df.astype({col_name:float,})
df.info()

TORQ_ptop
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 752929 entries, 2016-06-01 00:00:13 to 2017-12-01 21:12:12
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   TORQ_ptop  752929 non-null  float64
dtypes: float64(1)
memory usage: 11.5 MB


In [4]:
df=df[173420:305915]

In [5]:
df.head()

,TORQ_ptop
0,
2016-10-01 00:05:21,21.0
2016-10-01 00:06:21,22.3
2016-10-01 00:07:21,22.2
2016-10-01 00:08:21,20.8
2016-10-01 00:09:21,19.3


In [6]:
df.index.month

Int64Index([10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
            ...
            12, 12, 12, 12, 12, 12, 12, 12, 12,  1],
           dtype='int64', name=0, length=132495)

In [7]:
def func_rms(rms_df,col_name,threshold,sample_value):
    rms_df['square_values'] = rms_df[col_name]**2
    rms_df.drop(columns=[col_name],inplace=True)
    rms_df=rms_df.resample(sample_value).mean()
    rms_df.columns=['RMS']
    rms_df['RMS'] = rms_df['RMS'].apply(lambda x : x**0.5)
    #print(rms_df)
    outlier_df=rms_df[rms_df['RMS']<=threshold]
    outlier_df.columns=['Non Running']
    outlier_df.index=pd.to_datetime(outlier_df.index)
    final_df=rms_df[:]
    final_df.drop(final_df[final_df['RMS'] <=threshold].index, inplace= True)
    final_df.dropna(inplace=True)
    #print(final_df[final_df['RMS']==0])
    fit1 = SimpleExpSmoothing(final_df['RMS']).fit(smoothing_level=0.2,optimized = False)
    #print(fit1.fittedvalues)
    final_df['Smooth RMS']=list(fit1.fittedvalues)
    final_df.drop(columns=['RMS'],inplace=True)
    rms_df=pd.concat([rms_df,final_df], axis=1, join='outer')
    rms_df['Smooth RMS']=rms_df['Smooth RMS'].interpolate('linear')
    rms_df=pd.concat([rms_df,outlier_df], axis=1, join='outer')
    rms_df.to_csv("TORQ\Data\\"+col_name+"_RMS.csv")  

    #plots
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=rms_df.index,y=rms_df['Smooth RMS'],mode='lines',name = 'Smooth RMS',line=dict(color="#ffae42")))
    fig.add_trace(go.Scatter(x=rms_df.index,y=rms_df['RMS'],mode='markers',name = 'RMS',line=dict(color="#5dd55d")))
    fig.add_trace(go.Scatter(x=rms_df.index,y=rms_df['Non Running'],mode='markers',name = 'Non Running',line=dict(color="#ff0000")))
    fig.update_layout(hovermode="x")
    fig.update_layout(xaxis=dict(showgrid=True),yaxis=dict(showgrid=True),title={'text':'RMS DATA','xanchor':'center','x':0.5})
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(title_text='Amplitude')
    fig.write_html("TORQ\Plots\\"+col_name+"_RMS.html")
    

In [8]:
def func_peak(peak_df,col_name,threshold,sample_value):
    peak_df=peak_df.resample(sample_value).apply(lambda x:abs(x).max())
    peak_df.columns=['peak']
    outlier_df=peak_df[peak_df['peak']<=threshold]
    outlier_df.columns=['Non Running']
    #outlier_df.index=pd.to_datetime(outlier_df.index)
    final_df=peak_df[:]
    final_df.drop(final_df[final_df['peak'] <=threshold].index, inplace= True)
    final_df.dropna(inplace=True)
    fit1 = SimpleExpSmoothing(final_df['peak']).fit(smoothing_level=0.2,optimized = False)
    #print(fit1.fittedvalues)
    final_df['Smooth peak']=list(fit1.fittedvalues)
    final_df.drop(columns=['peak'],inplace=True)
    peak_df=pd.concat([peak_df,final_df], axis=1, join='outer')
    peak_df['Smooth peak']=peak_df['Smooth peak'].interpolate('linear')
    peak_df=pd.concat([peak_df,outlier_df], axis=1, join='outer')
    peak_df.to_csv("TORQ\Data\\"+col_name+"_peak.csv")  

    #plots
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=peak_df.index,y=peak_df['Smooth peak'],mode='lines',name = 'Smooth Peak',line=dict(color="#ffae42")))
    fig.add_trace(go.Scatter(x=peak_df.index,y=peak_df['peak'],mode='markers',name = 'Peak',line=dict(color="#5dd55d")))
    fig.add_trace(go.Scatter(x=peak_df.index,y=peak_df['Non Running'],mode='markers',name = 'Non Running',line=dict(color="#ff0000")))
    fig.update_layout(hovermode="x")
    fig.update_layout(xaxis=dict(showgrid=True),yaxis=dict(showgrid=True),title={'text':'PEAK DATA','xanchor':'center','x':0.5})
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(title_text='Amplitude')
    fig.write_html("TORQ\Plots\\"+col_name+"_peak.html")
    

In [9]:
def func_kurt(kurt_df,col_name,threshold,sample_value):
    kurt_df=kurt_df.resample(sample_value).apply(lambda x:kurtosis(x,fisher=False))
    kurt_df.columns=['kurtosis']
    outlier_df=kurt_df[kurt_df['kurtosis']<=threshold]
    outlier_df.columns=['Non Running']
    #outlier_df.index=pd.to_datetime(outlier_df.index)
    final_df=kurt_df[:]
    final_df.drop(final_df[final_df['kurtosis'] <=threshold].index, inplace= True)
    final_df.dropna(inplace=True)
    fit1 = SimpleExpSmoothing(final_df['kurtosis']).fit(smoothing_level=0.2,optimized = False)
    #print(fit1.fittedvalues)
    final_df['Smooth kurtosis']=list(fit1.fittedvalues)
    final_df.drop(columns=['kurtosis'],inplace=True)
    kurt_df=pd.concat([kurt_df,final_df], axis=1, join='outer')
    kurt_df['Smooth kurtosis']=kurt_df['Smooth kurtosis'].interpolate('linear')
    kurt_df=pd.concat([kurt_df,outlier_df], axis=1, join='outer')
    kurt_df.to_csv("TORQ\Data\\"+col_name+"_kurtosis.csv")  

    #plots
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=kurt_df.index,y=kurt_df['Smooth kurtosis'],mode='lines',name = 'Smooth Kurtosis',line=dict(color="#ffae42")))
    fig.add_trace(go.Scatter(x=kurt_df.index,y=kurt_df['kurtosis'],mode='markers',name = 'Kurtosis',line=dict(color="#5dd55d")))
    fig.add_trace(go.Scatter(x=kurt_df.index,y=kurt_df['Non Running'],mode='markers',name = 'Non Running',line=dict(color="#ff0000")))
    fig.update_layout(hovermode="x")
    fig.update_layout(xaxis=dict(showgrid=True),yaxis=dict(showgrid=True),title={'text':'KURTOSIS DATA','xanchor':'center','x':0.5})
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(title_text='Amplitude')
    fig.write_html("TORQ\Plots\\"+col_name+"_kurtosis.html")
    

In [10]:
func_rms(df[:],col_name,1,'D')

C:\Users\2000139\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [11]:
func_peak(df[:],col_name,1,'D')

C:\Users\2000139\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [12]:
func_kurt(df[:],col_name,0,'D')